In [33]:
import tensorflow as tf

tf.__version__

'2.1.0'

In [2]:
import pandas as pd
import os, shutil
import cv2
base_dir = 'D:\Final\code\LAB\keras\Distracted Driver Detection\imgs'

driver_details = pd.read_csv('driver_imgs_list.csv',na_values='na')
print(driver_details.head())

  subject classname            img
0    p002        c0  img_44733.jpg
1    p002        c0  img_72999.jpg
2    p002        c0  img_25094.jpg
3    p002        c0  img_69092.jpg
4    p002        c0  img_92629.jpg


In [3]:
train_image = []
image_label = []


for i in range(10):
    print('folder C',i)
    imgs = os.listdir(base_dir+"/train/c"+str(i))
    for j in range(len(imgs)):
        img_name = base_dir+"/train/c"+str(i)+"/"+imgs[j]
        img = cv2.imread(img_name)
        img = img[:,120:-30] # Cropping:去掉部分左邊和右邊一點點
        img = cv2.resize(img,(140,140)) # Resizing
        label = i
        driver = str(driver_details[driver_details['img'] == imgs[j]]['subject'].values[0])
        train_image.append([img,label,driver])
        image_label.append(i)

folder C 0
folder C 1
folder C 2
folder C 3
folder C 4
folder C 5
folder C 6
folder C 7
folder C 8
folder C 9


In [28]:
image_label[-1]

9

In [4]:
#import pickle
#
#with open('image_label.pickle','wb') as f1:
#    pickle.dump(image_label,f1)

In [5]:
#with open('train_image.pickle','wb') as f2:
 #   pickle.dump(train_image,f2)

In [6]:
print(len(train_image), len(image_label))

22424 22424


In [7]:
import random
random.shuffle(train_image)

In [8]:
drivers = []
for F,L,D in train_image:
    drivers.append(D)
drivers = list(set(drivers))

In [9]:
len(drivers)

26

In [10]:
drivers[:4]

['p041', 'p039', 'p045', 'p072']

# Split on subjects

In [29]:
x_train = []
y_train = []
x_val = []
y_val = []
drivers_train = []
drivers_val = []

for F,L,D in train_image:
    if D in drivers[:4]:
        drivers_val.append(D)
        x_val.append(F)
        y_val.append(L)
    else:
        drivers_train.append(D)
        x_train.append(F)
        y_train.append(L)

In [30]:
print(len(x_train))
print(len(x_val))

20098
2326


In [31]:
import numpy as np

# Converting images to nparray
x_train = np.array(x_train).reshape(-1,140,140,3) #等於.reshape(19181,140,140,3)
x_val = np.array(x_val).reshape(-1,140,140,3)

print (x_train.shape)

(20098, 140, 140, 3)


In [32]:
# one_hot_encoding
y_train = tf.keras.utils.to_categorical(y_train)
y_val = tf.keras.utils.to_categorical(y_val)

print(y_train.shape)

(20098, 10)


In [15]:
y_train[1]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [22]:
from tensorflow.keras.applications.xception import Xception

xception_input = tf.keras.layers.Input(shape = (140, 140, 3), name = 'Image_input')

model_xception_conv = Xception(weights= 'imagenet', include_top=False, input_shape= (140,140,3))
model_xception_conv.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 140, 140, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 69, 69, 32)   864         input_5[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 69, 69, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 69, 69, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [24]:
output_xception_conv = model_xception_conv(xception_input)

#Add the fully-connected layers 

x = tf.keras.layers.Flatten(name='flatten')(output_xception_conv)
x = tf.keras.layers.Dense(10, activation='softmax', name='predictions')(x)


xception_pretrained = tf.keras.Model(inputs = xception_input, outputs = x)


xception_pretrained.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Image_input (InputLayer)     [(None, 140, 140, 3)]     0         
_________________________________________________________________
xception (Model)             (None, 5, 5, 2048)        20861480  
_________________________________________________________________
flatten (Flatten)            (None, 51200)             0         
_________________________________________________________________
predictions (Dense)          (None, 10)                512010    
Total params: 21,373,490
Trainable params: 21,318,962
Non-trainable params: 54,528
_________________________________________________________________


In [25]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(#壓縮在[-1~1]
    featurewise_center=True,
    samplewise_center=True,
    rescale = 2./255
)

data_generator = datagen.flow(x_train, y_train, batch_size = 32)
xception_model = model.fit_generator(data_generator,
                             steps_per_epoch = len(x_train)/32,
                             epochs = 10,
                             validation_data = (x_val,y_val),
                            )

Instructions for updating:
Please use Model.fit, which supports generators.


C:\Users\user\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


  ...
    to  
  ['...']
Train for 628.0625 steps, validate on 2326 samples
Epoch 1/10


C:\Users\user\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


629/628 [==============================] - 732s 1s/step - loss: 1.7176 - accuracy: 0.3606 - val_loss: 10623.5440 - val_accuracy: 0.0993
Epoch 2/10
  9/628 [..............................] - ETA: 9:54 - loss: 0.3947 - accuracy: 0.8906 

KeyboardInterrupt: 

In [37]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

model_inception3_conv = InceptionV3(weights= 'imagenet', include_top=False, input_shape= (140,140,3))
model_inception3_conv.summary()

87916544/87910968 [==============================] - 107s 1us/step
Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 140, 140, 3) 0                                            
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 69, 69, 32)   864         input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_12 (BatchNo (None, 69, 69, 32)   96          conv2d_12[0][0]                  
__________________________________________________________________________________________________
activation (Activation)         (None, 69, 69, 32)   0           batch_normalization_12[0][0]     
____________________

In [40]:
inputs = tf.keras.layers.Input(shape = (140, 140, 3), name = 'Image_input')

output_inception3_conv = model_inception3_conv(inputs)

#Add the fully-connected layers 

x = tf.keras.layers.Flatten(name='flatten')(output_inception3_conv)
outputs = tf.keras.layers.Dense(10, activation='softmax', name='predictions')(x)


inception3_pretrained = tf.keras.Model(inputs = inputs, outputs = outputs)


inception3_pretrained.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Image_input (InputLayer)     [(None, 140, 140, 3)]     0         
_________________________________________________________________
inception_v3 (Model)         (None, 3, 3, 2048)        21802784  
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
predictions (Dense)          (None, 10)                184330    
Total params: 21,987,114
Trainable params: 21,952,682
Non-trainable params: 34,432
_________________________________________________________________
